<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/master/Word2Vec_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf

# Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


#Load FAANG Data

In [ ]:
import numpy as np
# data = pd.read_csv('/content/drive/My Drive/Data/Combined_FAANG_percentage_2.2.csv')
data = pd.read_csv('/content/drive/My Drive/Data/Combined_FAANG_binary_previous.csv')

print("Number of examples {}".format(len(data)))

Number of examples 2566858


Limiting dataset to 1 year

In [ ]:
# data = data[data["symbol"].isin(['AAPL', 'FB', 'AMZN', 'NFLX', 'GOOGL'])]

data.drop(data[data['Date'] < '2019-07-20'].index, inplace = True)
data

,symbol,message,datetime,user,message_id,Date,Time,label
3,AAPL,aapl,2020-01-27 07:07:03,1229493.0,191978042.0,2020-01-27,07:07:03,0
8,AAPL,qq became euphoric calls exp week aiming ath f...,2020-05-13 02:13:00,2250451.0,212222428.0,2020-05-13,02:13:00,0
9,AAPL,spy novices like davey day trader lose money s...,2020-06-24 11:12:09,543250.0,222404886.0,2020-06-24,11:12:09,0
10,AAPL,today aapl shows buy signal ta short term tech...,2019-09-11 09:33:43,700679.0,176835918.0,2019-09-11,09:33:43,1
11,AAPL,aapl let melt begin,2020-01-03 16:46:16,741099.0,188910094.0,2020-01-03,16:46:16,0
...,...,...,...,...,...,...,...,...
2566840,NFLX,hope aside nyt putting also cnbc puts nflx take,2019-09-21 21:49:00,2807876.0,178253955.0,2019-09-21,21:49:00,0
2566841,NFLX,amd msft nflx everything beautiful 39 way,2020-03-06 06:06:03,911299.0,198589415.0,2020-03-06,06:06:03,0
2566848,NFLX,nflx full move key support,2020-03-12 16:52:43,677915.0,199933357.0,2020-03-12,16:52:43,0
2566849,NFLX,spy spx nflx nvda virtually volume today absen...,2019-10-14 18:16:28,55818.0,180328889.0,2019-10-14,18:16:28,1


Dropping Neutral (wherever required)

In [ ]:
# data.drop(data[data['label'] == 0].index, inplace = True) 
# data["label"].replace({-1: 0}, inplace=True)

In [ ]:
data = data.sample(frac=1)
data

,symbol,message,datetime,user,message_id,Date,Time,label
2078476,NFLX,nflx back 335,2019-07-26 21:10:36,2125598.0,171955278.0,2019-07-26,21:10:36,1
2168255,NFLX,8 latest short seller targets bx v nflx fdx bmy,2019-09-30 16:56:52,7108.0,179047530.0,2019-09-30,16:56:52,1
1643232,AMZN,amzn disney shares fall earnings miss,2019-08-06 20:19:47,316034.0,173169674.0,2019-08-06,20:19:47,1
933245,AAPL,tsla aapl spy amd amzn lol course,2020-03-09 20:52:49,277257.0,199186640.0,2020-03-09,20:52:49,0
2339611,NFLX,nflx long long time since rsi low,2019-09-19 13:55:28,391844.0,178015795.0,2019-09-19,13:55:28,0
...,...,...,...,...,...,...,...,...
521672,AAPL,spy djia ba aapl 39 weird people living payche...,2020-03-23 03:17:34,2266176.0,202033813.0,2020-03-23,03:17:34,0
2141752,NFLX,nflx bounced support closed resistance expecti...,2019-07-29 03:08:46,2185664.0,172058434.0,2019-07-29,03:08:46,0
1243666,AMZN,amzn new article amazon put tesla notice,2020-07-21 13:07:59,1555408.0,229495210.0,2020-07-21,13:07:59,0
907693,AAPL,aapl breaks 22 5 huge squeeze imminent,2019-09-30 14:32:49,2110951.0,179027010.0,2019-09-30,14:32:49,1


In [ ]:
data = data.dropna(subset=['message'])  
data

,symbol,message,datetime,user,message_id,Date,Time,label
2078476,NFLX,nflx back 335,2019-07-26 21:10:36,2125598.0,171955278.0,2019-07-26,21:10:36,1
2168255,NFLX,8 latest short seller targets bx v nflx fdx bmy,2019-09-30 16:56:52,7108.0,179047530.0,2019-09-30,16:56:52,1
1643232,AMZN,amzn disney shares fall earnings miss,2019-08-06 20:19:47,316034.0,173169674.0,2019-08-06,20:19:47,1
933245,AAPL,tsla aapl spy amd amzn lol course,2020-03-09 20:52:49,277257.0,199186640.0,2020-03-09,20:52:49,0
2339611,NFLX,nflx long long time since rsi low,2019-09-19 13:55:28,391844.0,178015795.0,2019-09-19,13:55:28,0
...,...,...,...,...,...,...,...,...
521672,AAPL,spy djia ba aapl 39 weird people living payche...,2020-03-23 03:17:34,2266176.0,202033813.0,2020-03-23,03:17:34,0
2141752,NFLX,nflx bounced support closed resistance expecti...,2019-07-29 03:08:46,2185664.0,172058434.0,2019-07-29,03:08:46,0
1243666,AMZN,amzn new article amazon put tesla notice,2020-07-21 13:07:59,1555408.0,229495210.0,2020-07-21,13:07:59,0
907693,AAPL,aapl breaks 22 5 huge squeeze imminent,2019-09-30 14:32:49,2110951.0,179027010.0,2019-09-30,14:32:49,1


In [ ]:
data.label.value_counts()

1    283541
0    231104
Name: label, dtype: int64

In [ ]:
print('Positive samples: %d of %d (%.2f%%)' % (data.label.sum(), len(data.label), (data.label.sum() / len(data.label) * 100.0)))

Positive samples: 283541 of 514645 (55.09%)


Splitting dataset into 90-10 

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, y_train, y_test = train_test_split(data['message'], data['label'], test_size=0.1, random_state=42)

In [ ]:
print("Number of training examples {}".format(len(X_train)))
print("Number of testing examples {}".format(len(X_test)))

Number of training examples 463180
Number of testing examples 51465


In [ ]:
y_test

612463     0
2057093    1
1344873    1
1649199    0
1192622    1
          ..
2340538    0
593263     0
1731787    1
1913447    1
27080      1
Name: label, Length: 51465, dtype: int64

Creating training and testing dataframes

In [ ]:
df_train = pd.DataFrame(X_train,columns=['message'])
df_train

,message
2047154,would say amzn 39 essentials roku nflx
1989285,nflx give
1839479,amazon looks ready 40 rally says market watche...
1457195,amzn independent bookstores get creative survi...
2009364,nflx first weekly close june 39 18 buying clim...
...,...
1728294,spy dis aapl tsla another fantastic testimonia...
1243445,amzn nothing better hours move amazon great in...
832551,aapl apple tv premium shows comparison 39 watc...
2088843,impressive aapl fb amzn nflx amp goog red appl...


In [ ]:
df_test = pd.DataFrame(X_test,columns=['message'])
df_test

,message
612463,whole rally made 5 dead cats day amd I good go...
2057093,nflx buyer valuation growth good metric high c...
1344873,v amzn aapl pypl 22 sec ago europe must wean g...
1649199,uavs backed amzn 10 bagger easy
1192622,aapl really hope smart people bought beautiful...
...,...
2340538,nflx thinking 229 eod 336 eow
593263,aapl let us
1731787,dis bounce coming
1913447,nflx looking like buy imo


In [ ]:
y_train = pd.DataFrame(y_train,columns=['label'])
y_train

,label
2047154,1
1989285,0
1839479,1
1457195,1
2009364,0
...,...
1728294,1
1243445,1
832551,1
2088843,0


#Import Libraries for CNN

In [ ]:
import numpy as np # linear algebra
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten, Dropout, Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model
from keras.callbacks import EarlyStopping
import gensim
import nltk
from nltk.tokenize import RegexpTokenizer
import re
import codecs


In [ ]:
EMBEDDING_DIM = 300 
MAX_VOCAB_SIZE = 175303 
MAX_SEQUENCE_LENGTH = 160 

#training params
batch_size = 64 
num_epochs = 2 

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
df_train["tokens"] = df_train["message"].apply(tokenizer.tokenize)

In [ ]:
df_train

,message,tokens
2047154,would say amzn 39 essentials roku nflx,"[would, say, amzn, 39, essentials, roku, nflx]"
1989285,nflx give,"[nflx, give]"
1839479,amazon looks ready 40 rally says market watche...,"[amazon, looks, ready, 40, rally, says, market..."
1457195,amzn independent bookstores get creative survi...,"[amzn, independent, bookstores, get, creative,..."
2009364,nflx first weekly close june 39 18 buying clim...,"[nflx, first, weekly, close, june, 39, 18, buy..."
...,...,...
1728294,spy dis aapl tsla another fantastic testimonia...,"[spy, dis, aapl, tsla, another, fantastic, tes..."
1243445,amzn nothing better hours move amazon great in...,"[amzn, nothing, better, hours, move, amazon, g..."
832551,aapl apple tv premium shows comparison 39 watc...,"[aapl, apple, tv, premium, shows, comparison, ..."
2088843,impressive aapl fb amzn nflx amp goog red appl...,"[impressive, aapl, fb, amzn, nflx, amp, goog, ..."


In [ ]:
df_test["tokens"] = df_test["message"].apply(tokenizer.tokenize)

In [ ]:
training_word_list = [word for tokens in df_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in df_train["tokens"]]
vocab_train = sorted(list(set(training_word_list)))
print("%s words total, with a vocabulary size of %s" % (len(training_word_list), len(vocab_train)))
print("Max sentence length is %s" % max(training_sentence_lengths))

6034876 words total, with a vocabulary size of 74460
Max sentence length is 789


In [ ]:
test_word_list = [word for tokens in df_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in df_test["tokens"]]
vocab_test = sorted(list(set(test_word_list)))
print("%s words total, with a vocabulary size of %s" % (len(test_word_list), len(vocab_test)))
print("Max sentence length is %s" % max(test_sentence_lengths))

665914 words total, with a vocabulary size of 28309
Max sentence length is 183


# Loading Word2Vec Model

In [ ]:
word2vec_path = "/content/drive/My Drive/Word2Vec/GoogleNews-vectors-negative300.bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_message, generate_missing=False):
    embeddings = clean_message['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

Creating embeddings

In [ ]:
training_embeddings = get_word2vec_embeddings(word2vec, df_train, generate_missing=True)

In [ ]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(df_train['message'].tolist())
training_sequences = tokenizer.texts_to_sequences(df_train["message"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

Found 74389 unique tokens.
(74390, 300)


In [ ]:
train_embedding_weights

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.04663086, -0.10058594, -0.06933594, ..., -0.02490234,
        -0.15429688,  0.14941406],
       [ 0.90292264,  0.53585593,  0.53837297, ...,  0.17690553,
         0.76411943,  0.4151133 ],
       ...,
       [-0.1328125 ,  0.15039062,  0.03857422, ..., -0.24316406,
        -0.11865234,  0.12792969],
       [-0.08105469,  0.03515625,  0.03515625, ..., -0.05493164,
        -0.06176758,  0.12353516],
       [ 0.28320312, -0.05883789,  0.08056641, ..., -0.22265625,
        -0.00367737,  0.20898438]])

In [ ]:
test_sequences = tokenizer.texts_to_sequences(df_test["message"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Building CNN model

In [ ]:
from keras.layers import concatenate
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate(convs, axis = 1)
    # add(mode='concat', concat_axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [ ]:
train_cnn_data

array([[   0,    0,    0, ..., 5469,   94,    3],
       [   0,    0,    0, ...,    0,    3,  268],
       [   0,    0,    0, ...,    2,   18,   59],
       ...,
       [   0,    0,    0, ...,    7,   22, 1124],
       [   0,    0,    0, ...,  420,  708,  415],
       [   0,    0,    0, ..., 3951,  262, 2338]], dtype=int32)

In [ ]:
df_train = train_cnn_data
y_tr = y_train.label.values

In [ ]:
y_tr

array([1, 0, 1, ..., 1, 0, 0])

In [ ]:
label_names=['label']

In [ ]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)), False)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 160, 300)     29863800    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 158, 128)     115328      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 157, 128)     153728      embedding[0][0]                  
_______________________________________________________________________________________

In [ ]:

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

Model Training

In [ ]:
hist = model.fit(df_train, y_tr, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/2
12951/12951 [==============================] - 2529s 195ms/step - loss: 0.6910 - acc: 0.5353 - val_loss: 0.6902 - val_acc: 0.5371
Epoch 2/2
12951/12951 [==============================] - 2503s 193ms/step - loss: 0.6901 - acc: 0.5373 - val_loss: 0.6895 - val_acc: 0.5371


In [ ]:
df_train

array([[    0,     0,     0, ...,   253, 12931, 12314],
       [    0,     0,     0, ...,    92,    61,    27],
       [    0,     0,     0, ...,    26,   445,   296],
       ...,
       [    0,     0,     0, ...,     0,     0,     3],
       [    0,     0,     0, ...,  2884,   138,     1],
       [    0,     0,     0, ...,  1961,  1523,  2049]], dtype=int32)

In [ ]:
y_predict_test = model.predict(test_cnn_data, batch_size=256, verbose=1)

400/400 [==============================] - 94s 235ms/step


In [ ]:
model.evaluate(test_cnn_data, y_test, verbose=0)

[0.6895556449890137, 0.5368821024894714]

In [ ]:
y_predict_test

array([[0.53484327],
       [0.53484327],
       [0.5911659 ],
       ...,
       [0.53601116],
       [0.53484327],
       [0.53484327]], dtype=float32)

In [ ]:
#y_pred = np.argmax(Y_pred, axis=1) - used for multiclass
y_pred = (y_predict_test > 0.53) * 1.0
# y_pred = y_pred.reshape(y_val.shape)
y_pred.sum()

96082.0

# Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred,zero_division=1))

              precision    recall  f1-score   support

           0       0.54      0.07      0.13     47417
           1       0.54      0.95      0.69     54909

    accuracy                           0.54    102326
   macro avg       0.54      0.51      0.41    102326
weighted avg       0.54      0.54      0.43    102326



In [ ]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

Confusion Matrix
[[12273  4675]
 [15347  8663]]
